In [2]:
%cd /Users/aguschin/Git/iterative/yolov5/

/Users/aguschin/Git/iterative/yolov5


In [31]:
import yaml
import subprocess


def expand_dict(dictionary):
    string = ""
    for key, value in dictionary.items():
        if isinstance(value, dict):
            assert "Nested dicts aren't supported"
        elif isinstance(value, list):
            # I supposed it should work some other way
            string = f"{string} --{key} " + " ".join(f"{v}" for v in value)
        elif value is None:
            pass
        # not sure False should always turn into skipping that arg
        elif value is False:
            pass
        elif value is True:
            string = f"{string} --{key}"
        # why single quotes? It there two cases which required this option to be selected by user, or they are interchargeable?
        elif value == "":
            string = f"{string} --{key} ''"
        else:
            string = f"{string} --{key} {value}"
    return string

In [32]:
with open("params.yaml") as f:
    params = yaml.safe_load(f)

params_string = expand_dict(params["train"])
params_string

" --weights '' --cfg models/yolov5s.yaml --data data/coco128.yaml --hyp data/hyp.scratch.yaml --epochs 1 --batch-size 16 --img-size 32 32 --bucket '' --device cpu --local_rank -1 --workers 8 --project runs/train --name exp --label-smoothing 0.0 --bbox_interval -1 --save_period -1 --artifact_alias latest"

In [33]:
run = subprocess.run(f"python train_mock.py {params_string}", shell=True, capture_output=True, check=False)
run.stdout, run.stderr

(b"{'weights': '', 'cfg': 'models/yolov5s.yaml', 'data': 'data/coco128.yaml', 'hyp': 'data/hyp.scratch.yaml', 'epochs': 1, 'batch_size': 16, 'img_size': [32, 32], 'rect': False, 'resume': False, 'nosave': False, 'notest': False, 'noautoanchor': False, 'evolve': False, 'bucket': '', 'cache_images': False, 'image_weights': False, 'device': 'cpu', 'multi_scale': False, 'single_cls': False, 'adam': False, 'sync_bn': False, 'local_rank': -1, 'workers': 8, 'project': 'runs/train', 'entity': None, 'name': 'exp', 'exist_ok': False, 'quad': False, 'linear_lr': False, 'label_smoothing': 0.0, 'upload_dataset': False, 'bbox_interval': -1, 'save_period': -1, 'artifact_alias': 'latest'}\n",
 b'')

In [34]:
run = subprocess.run(f"python train_mock.py", shell=True, capture_output=True, check=False)
run.stdout, run.stderr

(b"{'weights': 'yolov5s.pt', 'cfg': '', 'data': 'data/coco128.yaml', 'hyp': 'data/hyp.scratch.yaml', 'epochs': 300, 'batch_size': 16, 'img_size': [640, 640], 'rect': False, 'resume': False, 'nosave': False, 'notest': False, 'noautoanchor': False, 'evolve': False, 'bucket': '', 'cache_images': False, 'image_weights': False, 'device': '', 'multi_scale': False, 'single_cls': False, 'adam': False, 'sync_bn': False, 'local_rank': -1, 'workers': 8, 'project': 'runs/train', 'entity': None, 'name': 'exp', 'exist_ok': False, 'quad': False, 'linear_lr': False, 'label_smoothing': 0.0, 'upload_dataset': False, 'bbox_interval': -1, 'save_period': -1, 'artifact_alias': 'latest'}\n",
 b'')

In [36]:
from copy import deepcopy

subprocess.run("cp dvc.yaml.save dvc.yaml", shell=True)

with open("params.yaml") as f:
    params = yaml.safe_load(f)

with open("dvc.yaml") as f:
    dvc = yaml.safe_load(f)

# dvc_copy = deepcopy(dvc)
for stage in dvc["stages"]:
    # hack for avoiding regexp, but this could be rewritten
    dvc["stages"][stage]["cmd"] = dvc["stages"][stage]["cmd"].replace("${" + stage + "}", expand_dict(params[stage]))
    
with open("dvc.yaml", "w") as f:
    yaml.safe_dump(dvc, f)

In [ ]:
run = subprocess.run(f"dvc repro", shell=True, capture_output=True, check=False)
run.stdout, run.stderr